## Imports

We'll begin by:
- Importing the necessary libraries
- Selecting models for embeddings search and question answering



In [2]:
# imports
import ast  # for converting embeddings saved as strings back to arrays
from openai import OpenAI # for calling the OpenAI API
import pandas as pd  # for storing text and embeddings data
import tiktoken  # for counting tokens
import os # for getting API token from env variable OPENAI_API_KEY
from scipy import spatial  # for calculating vector similarities for search

# models
EMBEDDING_MODEL = "text-embedding-3-large"
GPT_MODEL = "gpt-4o"

client = OpenAI()


## 1. Prepare search data

In [3]:
# download pre-chunked text and pre-computed embeddings
df = pd.read_csv("artisan.csv")

In [4]:
# convert embeddings from CSV str type back to list type
df['embedding'] = df['embedding'].apply(ast.literal_eval)

In [5]:
# the dataframe has two columns: "text" and "embedding"
df

,text,embedding
0,Artisan\n\nArtisan,"[-0.01277793782389468, -0.003185376819509296, ..."
1,Artisan\n\nArtisan,"[-0.01277793782389468, -0.003185376819509296, ..."
2,How To Setup New Domains & Email Accounts | Ar...,"[-0.015241750667813539, -0.016197075238963993,..."
3,"How To Set Up Your DKIM, SPF and DMARC | Artis...","[0.014848935559158742, -0.04646190381222231, -..."
4,How To Add Variables To Your Email Templates |...,"[0.010927064541469777, -0.03749197097060811, -..."
...,...,...
95,Talk to Sales | Use Artisan to Supercharge You...,"[-0.015319721481669218, -0.020032235971266715,..."
96,Talk to Sales | Use Artisan to Supercharge You...,"[-0.01544967977944185, -0.020016565130510023, ..."
97,Terms of Use - Artisan\n\nTerms of Use - Artis...,"[-0.0032217373928867302, -0.05428168057532449,..."
98,Terms of Use - Artisan\n\nTerms of Use - Artis...,"[-0.0032021576198882047, -0.05418228285275306,..."


## 2. Search

- Takes a user query and a dataframe with text & embedding columns
- Embeds the user query with the OpenAI API
- Uses distance between query embedding and text embeddings to rank the texts
- Returns two lists:
    - The top N texts, ranked by relevance
    - Their corresponding relevance scores

In [6]:
# search function
def strings_ranked_by_relatedness(
    query: str,
    df: pd.DataFrame,
    relatedness_fn=lambda x, y: 1 - spatial.distance.cosine(x, y),
    top_n: int = 100
) -> tuple[list[str], list[float]]:
    """Returns a list of strings and relatednesses, sorted from most related to least."""
    query_embedding_response = client.embeddings.create(
        model=EMBEDDING_MODEL,
        input=query,
    )
    query_embedding = query_embedding_response.data[0].embedding
    strings_and_relatednesses = [
        (row["text"], relatedness_fn(query_embedding, row["embedding"]))
        for i, row in df.iterrows()
    ]
    strings_and_relatednesses.sort(key=lambda x: x[1], reverse=True)
    strings, relatednesses = zip(*strings_and_relatednesses)
    return strings[:top_n], relatednesses[:top_n]


In [7]:
# examples
strings, relatednesses = strings_ranked_by_relatedness("Does Ava take care of email warmup?", df, top_n=5)
for string, relatedness in zip(strings, relatednesses):
    print(f"{relatedness=:.3f}")
    display(string)

relatedness=0.664


'What Is Email Warmup? | Artisan AI Help Center\n\nWhat Is Email Warmup? | Artisan AI Help Center Skip to main content Artisan AI Help Center Back To App English ; English Back To App English ; English Table of contents All Collections Artisan Sales What Is Email Warmup? What Is Email Warmup? Here we explain what email warmup is, how it works, why we use it and what to expect! Updated over a week ago Table of contents Email warmup works by gradually increasing the volume of emails sent from a new account over a period of time. This establishes a positive reputation with Internet Service Providers (ISPs), which improves deliverability and reduces the chance of emails getting flagged as spam. Given our recommendation to register new email accounts on our platform, we provide a built-in email warmup service. This ensures your mailboxes are prepared to handle your outbound email campaigns effectively. How Does Email Warmup Work? Warmup is an ongoing process that happens when you do cold ou

relatedness=0.612


"Help! Ava Is Sending Strange Messages From My Email | Artisan AI Help Center\n\nHelp! Ava Is Sending Strange Messages From My Email | Artisan AI Help Center Skip to main content Artisan AI Help Center Back To App English ; English Back To App English ; English All Collections Artisan Sales Help! Ava Is Sending Strange Messages From My Email Help! Ava Is Sending Strange Messages From My Email Is Ava sending emails that you don't recognize from your inbox? You haven't been hacked! It's our email warmup service. Updated over a week ago If you notice odd messages in your sent folder, don’t panic. Chances are, this is all part of our warmup email feature, which establishes your domain reputation to ensure messages don’t get filtered to spam. You can learn more here: https://support.artisan.co/en/articles/9191300-what-is-email-warmup During the warmup period, Ava sends fake warm interactions to dummy accounts to balance cold outbound. There is no need to be concerned about these strange ema

relatedness=0.604


"Ava Isn’t Sending Out My Emails. Why? | Artisan AI Help Center\n\nAva Isn’t Sending Out My Emails. Why? | Artisan AI Help Center Skip to main content Artisan AI Help Center Back To App English ; English Back To App English ; English Table of contents All Collections Artisan Sales Ava Isn’t Sending Out My Emails. Why? Ava Isn’t Sending Out My Emails. Why? Several factors could be preventing Ava from sending your emails. Read this article to identify what the issue might be. Updated over a week ago Table of contents We’re sorry to hear Ava has not been sending out your emails. There are several ways to troubleshoot the problem. By going over the steps below, we’re confident you’ll get Ava back up and running! Are You in the Warmup Period? Are you within the first three weeks of your account? If so, you may still be in the warmup period. Our built-in email warmup service helps establish your domain reputation to ensure Ava's emails do not get filtered to spam. During this time, Ava gradu

relatedness=0.585


"Use Our Email Warmup Tool to Optimize Your Deliverability\n\nUse Our Email Warmup Tool to Optimize Your Deliverability Contact Sales Login Contact Sales Products Solutions Resources Pricing Enterprise Built-In Email Warmup to Protect Your Domain Boost your sender reputation and sidestep spam filters with our AI-driven email warmup. Phone Get Started Features We Do Everything To Ensure Your Email Is Received We have a comprehensive strategy in place to maintain your email account health. By using your inbox to interact with real people, we ensure sustained, long-term deliverability. Automatically Remove Emails From Spam Artisan helps ensure your messages avoid the spam folder by opening them, marking them as important, and significantly improving your email reputation. Tailored Email Warmup Content Every email sent through Artisan's email warmup tool is fully personalized, enhancing deliverability and demonstrating that you are a genuine sender. Intelligent Email Warmup Strategy When y

relatedness=0.550


"Getting Started with Artisan Sales | Artisan AI Help Center\n\nGetting Started with Artisan Sales | Artisan AI Help Center Skip to main content Artisan AI Help Center Back To App English ; English Back To App English ; English Table of contents All Collections Artisan Sales Getting Started with Artisan Sales Getting Started with Artisan Sales How to get started with the Artisan Sales platform when you first sign up for an account. Updated over a week ago Table of contents Welcome to our platform! We’re excited to have you here. The Artisan Sales platform is designed to streamline your outbound workflow, with all the tools you need in one place. We have built-in email warmup, bounce testing, and mailbox health monitoring to ensure your deliverability is optimized. We also have an analytics dashboard, so you can check which campaigns and playbooks are doing the best. And of course, we have our AI BDR Ava, who automates all the manual parts of outbound for you! Once you've set up your ca

## 3. Ask

With the search function above, we can now automatically retrieve relevant knowledge and insert it into messages to GPT.

- Takes a user query
- Searches for text relevant to the query
- Stuffs that text into a message for GPT
- Sends the message to GPT
- Returns GPT's answer

In [22]:
def num_tokens(text: str, model: str = GPT_MODEL) -> int:
    """Return the number of tokens in a string."""
    encoding = tiktoken.encoding_for_model(model)
    return len(encoding.encode(text))


def query_message(
    query: str,
    df: pd.DataFrame,
    model: str,
    token_budget: int
) -> str:
    """Return a message for GPT, with relevant source texts pulled from a dataframe."""
    strings, relatednesses = strings_ranked_by_relatedness(query, df)
    introduction = 'Use the below articles on Artisan to answer the subsequent question."'
    question = f"\n\nQuestion: {query}"
    message = introduction
    for string in strings:
        next_article = f'\n\nArtisan article:\n"""\n{string}\n"""'
        if (
            num_tokens(message + next_article + question, model=model)
            > token_budget
        ):
            break
        else:
            message += next_article
    return message + question


def ask(
    query: str,
    df: pd.DataFrame = df,
    model: str = GPT_MODEL,
    token_budget: int = 4096 - 500,
    print_message: bool = False,
) -> str:
    """Answers a query using GPT and a dataframe of relevant texts and embeddings."""
    message = query_message(query, df, model=model, token_budget=token_budget)
    if print_message:
        print(message)
    messages = [
        {"role": "system", "content": "You are Ava, Artisan's AI BDR. Use sources to provide informed answers to user questions about the product. The user won't see the retrieved sources, so don't include phrases like 'according to the articles.'"},
        {"role": "user", "content": message},
    ]
    response = client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=0
    )
    response_message = response.choices[0].message.content
    return response_message



### Example questions

In [23]:
ask('What can you do?')

'Artisan offers a comprehensive AI-first platform designed to streamline and supercharge your outbound sales workflow. Here are the key features and capabilities:\n\n1. **AI BDR Ava**: Ava automates the manual parts of outbound sales, including finding leads, conducting research, and writing personalized emails for you to review and send.\n\n2. **Email Warmup and Deliverability**: The platform includes built-in email warmup, bounce testing, and mailbox health monitoring to ensure optimal email deliverability.\n\n3. **Analytics Dashboard**: You can track the performance of your campaigns and playbooks through an integrated analytics dashboard.\n\n4. **Lead Generation**: Ava populates your outbound CRM with leads by scraping a database of over 300 million B2B contacts, based on the target customer persona information you provide.\n\n5. **Email Personalization**: Ava scrapes leads’ websites to gather information necessary for crafting hyper-personalized messages.\n\n6. **Email Sending**: 

### Troubleshooting answers

To see whether a mistake is from a lack of relevant source text (i.e., failure of the search step) or a lack of reasoning reliability (i.e., failure of the ask step), you can look at the text GPT was given by setting `print_message=True`.

In [10]:
# set print_message=True to see the source text GPT was working off of
ask('Will my CRMs be automatically synced if a lead responds to my email??', print_message=True)

Use the below articles on Artisan to answer the subsequent question."

Artisan article:
"""
How Do I Upload a CSV File of My Own Leads? | Artisan AI Help Center

How Do I Upload a CSV File of My Own Leads? | Artisan AI Help Center Skip to main content Artisan AI Help Center Back To App English ; English Back To App English ; English Table of contents All Collections Artisan Sales How Do I Upload a CSV File of My Own Leads? How Do I Upload a CSV File of My Own Leads? Learn how to upload a CSV file of your own leads for Ava to draft personalized emails for. Updated over a week ago Table of contents Watch the video tutorial on Youtube here: https://youtu.be/Q2cvz48wdKg?feature=shared If you would like Ava to contact a list of leads you’ve generated yourself, you can upload a CSV file with the leads included. Doing so is a simple process. All you need to do is click on "Upload CSV" button in the Target Customer Persona section of your Campaign Settings: Once you upload your file, Ava shoul

'No, your CRMs will not be automatically synced if a lead responds to your email. However, Artisan does offer integrations with Salesforce and HubSpot, which allow you to export your engaged leads and ensure you don’t reach out to anyone who’s already in your CRM. You would need to set up these integrations to manage your leads effectively.'

Knowing that this mistake was due to imperfect reasoning in the ask step, rather than imperfect retrieval in the search step, let's focus on improving the ask step.

The easiest way to improve results is to use a more capable model, such as `GPT-4`. Let's try it.